In [ ]:
import torch 
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from API_utils.dataset import API_Class


In [ ]:
DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/train.xlsx"
TEST_DATASET_PATH = "C:/Users/asus/Desktop/API/dataset/test.xlsx"
SAVE_MODEL_PATH = "C:/Users/asus/Desktop/API/Model/one.pth"
DEVICE= torch.device(0)
EPOCH = 50

In [ ]:
train_data = API_Class(DATASET_PATH)
test_data = API_Class(TEST_DATASET_PATH)

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NeuralNet(nn.Module):
    def __init__(self, lrate = 0.001, loss_fn = nn.CrossEntropyLoss()):
        super(NeuralNet, self).__init__()


        self.conv1 = nn.Sequential(nn.Conv1d(1,8,kernel_size=1),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv2 = nn.Sequential(nn.Conv1d(8,16,kernel_size=1),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )
        self.conv3 = nn.Sequential(nn.Conv1d(1,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.LeakyReLU()
        )
        self.conv4 = nn.Sequential(nn.Conv1d(8,16,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(16),
                                   nn.LeakyReLU()
        )

        self.conv5 = nn.Sequential(nn.Conv1d(16,8,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(8),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )
        self.conv6 = nn.Sequential(nn.Conv1d(8,1,kernel_size=3,stride=2),
                                   nn.BatchNorm1d(1),
                                   nn.MaxPool1d(kernel_size=2),
                                   nn.LeakyReLU()
        )

        self.conv7 = nn.Linear(16,2)
        

        # self.conv1=nn.Conv1d(1, 1, kernel_size=3, stride=1, padding=0, dilation=1, groups=1, bias=True)
        # self.batchnorm=nn.BatchNorm1d(num_features=1),
        # self.Relu=nn.ReLU()
        # self.pool=nn.MaxPool1d(kernel_size=3)
        # self.fc1=nn.Linear(64*4*4, 64)
        # self.dropout=nn.Dropout()
        # self.fc2 = nn.Linear(64, 10)

        self.loss_fn = loss_fn
        self.lrate=lrate        
        self.optimizer=optim.SGD(self.parameters(),self.lrate,momentum=0.9)
    def forward(self, rna, protein):
        """Performs a forward pass through your neural net (evaluates f(x)).

        @param x: an (N, in_size) Tensor
        @return y: an (N, out_size) Tensor of output from the network
        """
        '''
        extractor for protein 
        '''
        
        rna = self.conv1(rna)
        rna = self.conv2(rna)

        protein = self.conv3(protein)
        
        protein = self.conv4(protein)

        x = torch.cat((rna,protein),dim=2)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = torch.squeeze(x,dim=0)
        return x



       
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        # x = self.pool(F.relu(self.conv3(x)))
        # x = torch.flatten(x, 1) # flatten all dimensions except batch      
        # print(x.shape)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # y = F.relu(self.fc3(x))        
        # return y
        # #return torch.ones(x.shape[0], 1)





In [ ]:
train_data_loader = DataLoader(train_data,batch_size=1,shuffle=True)
Net = NeuralNet()
optimizer = optim.Adam(params=Net.parameters(),lr=0.001)
loss_fn = nn.CrossEntropyLoss()


In [30]:
for epoch in range(EPOCH):
    for data in tqdm(train_data_loader):
        api_input, api_label = data
        rna_input = api_input[:,0:64]
        protein_input = api_input[:,64:909]
        rna_input = torch.unsqueeze(rna_input,dim=0)
        protein_input = torch.unsqueeze(protein_input,dim=0)
        #api_label = torch.unsqueeze(api_label,dim=0)
        rna_input = rna_input.to(dtype=torch.float32)
        protein_input = protein_input.to(dtype=torch.float32)
        output = Net(rna_input,protein_input)

        # print(output.shape)
        # print(api_label.shape)
        optimizer.zero_grad()     
        Loss = loss_fn(output, api_label)  
        Loss.backward()  
        optimizer.step() 
    
torch.save(Net, SAVE_MODEL_PATH)
print('CSX')

100%|██████████| 2320/2320 [22:48<00:00,  1.43it/s]


CSX


In [31]:
model=torch.load(SAVE_MODEL_PATH)

In [33]:
test_data_loader = DataLoader(test_data,batch_size=1,shuffle=True)
acc=0
count=0
for idx, data in enumerate(train_data_loader):
    api_input, api_label = data
    count+=1
    rna_input = api_input[:,0:64]
    protein_input = api_input[:,64:909]
    rna_input = torch.unsqueeze(rna_input,dim=0)
    protein_input = torch.unsqueeze(protein_input,dim=0)
    #api_label = torch.unsqueeze(api_label,dim=0)
    rna_input = rna_input.to(dtype=torch.float32)
    protein_input = protein_input.to(dtype=torch.float32)
    output = model(rna_input,protein_input)
    if(torch.argmax(output)==api_label[0]):
        acc+=1

    
print(acc/count)



0.75
